In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import timeit

import os
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import  confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from matplotlib.pyplot import figure

/home/anwer/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 1. Data Load

In [3]:
dc = {}
l = 0 
for i in os.listdir("/home/anwer/Desktop/cls_embed/"):
    path = os.path.join("/home/anwer/Desktop/cls_embed/",i)
    with open(path,'rb') as handle:
        d = pickle.load(handle)
    l = l+1
   
    dc.update(d)
    if l == 7:
        break

In [4]:
train = pd.read_csv("../../csv_files_new_ppi/training_and_test_set/combined_prot_a_prot_b//combined_train_only_name.csv")
test =  pd.read_csv("../../csv_files_new_ppi/training_and_test_set/combined_prot_a_prot_b//combined_test_only_name.csv")

In [5]:
def return_embed(prot_name):
    try:
        return dc[prot_name]
    except:
        return np.nan
train['embed_vec'] = train["Protein_name"].apply(return_embed)
test['embed_vec'] = test["Protein_name"].apply(return_embed)

train = train.dropna()
test = test.dropna()

In [6]:
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
val = test.iloc[50000:]
test = test.iloc[0:50000]
test.label.value_counts()

0    45434
1     4566
Name: label, dtype: int64

In [7]:
val_neg = val[val.label == 0][0:6163]
val_pos = val[val.label == 1][0:6163]
val = pd.concat([val_neg,val_pos])
val = val.sample(frac=1).reset_index(drop=True)

In [8]:
print(val.label.value_counts())

val.head()

0    6163
1    5361
Name: label, dtype: int64


,Protein_name,label,embed_vec
0,Prot_1291389,0,"[0.086794764, 0.013156574, 0.0744264, 0.035471..."
1,Prot_1293446,1,"[0.04804376, -0.0044574244, 0.07434094, 0.0169..."
2,Prot_1020546,0,"[0.104832344, 0.002876829, 0.057814967, 0.0509..."
3,Prot_1169728,0,"[0.05599065, 0.0009083219, 0.11614447, 0.01728..."
4,Prot_1273451,0,"[0.08105801, 0.01175717, 0.10160175, 0.0184878..."


In [9]:
print(test.label.value_counts())
test.head()

0    45434
1     4566
Name: label, dtype: int64


,Protein_name,label,embed_vec
0,Prot_1044787,0,"[0.06247573, -0.036905456, 0.05860683, 0.04706..."
1,Prot_1119375,0,"[0.094629884, 0.030718494, 0.1497275, 0.017708..."
2,Prot_1217529,0,"[0.08543196, -0.004241375, 0.062733404, 0.0195..."
3,Prot_1282723,1,"[0.0911243, -0.05816635, 0.049549427, -0.00700..."
4,Prot_1266332,0,"[0.024641423, 0.024009382, 0.11739512, 0.04144..."


In [10]:
print(train.label.value_counts())
train.head()

1    113334
0    113292
Name: label, dtype: int64


,Protein_name,label,embed_vec
0,Prot_1125492,0,"[0.04241518, 0.031186987, 0.11757413, 0.003731..."
1,Prot_1368020,0,"[0.08584418, -0.005175212, 0.112279445, 0.0255..."
2,Prot_1272998,1,"[0.07758383, -0.0066675195, 0.1364541, 0.03598..."
3,Prot_1164555,1,"[0.07059293, 0.03491224, 0.09695114, 0.0062628..."
4,Prot_1006280,0,"[0.12699686, -0.003101216, 0.10533013, 0.03658..."


## 1.1 Dataloader

In [11]:
train_features = []
train_label = []
test_features = []
test_label =[]
val_features = []
val_label = []
for i in tqdm(range(len(train))):
    train_features.append(np.array(train.iloc[i].embed_vec))
    train_label.append(np.array(train.iloc[i].label))
    
for i in tqdm(range(len(test))):
    test_features.append(np.array(test.iloc[i].embed_vec))
    test_label.append(np.array(test.iloc[i].label))  
for i in tqdm(range(len(val))):
    
    val_features.append(np.array(val.iloc[i].embed_vec))
    val_label.append(np.array(val.iloc[i].label))

100%|██████████████████████████████████████████████████| 11524/11524 [00:01<00:00, 6942.94it/s]


In [13]:
train_features = np.array(train_features)
train_label = np.array(train_label)

test_features = np.array(test_features)
test_label = np.array(test_label)

val_features = np.array(val_features)
val_label = np.array(val_label)

In [14]:
val_label

array([0, 1, 0, ..., 0, 0, 0])

In [15]:
class Data(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [16]:
train_data = Data(torch.FloatTensor(train_features), 
                       torch.FloatTensor(train_label))

test_data = Data(torch.FloatTensor(test_features), 
                       torch.FloatTensor(test_label))

val_data = Data(torch.FloatTensor(val_features), 
                       torch.FloatTensor(val_label))

In [17]:
train_loader = DataLoader(dataset=train_data, batch_size=2048*10, shuffle=True,drop_last=True)
test_loader = DataLoader(dataset=test_data, batch_size=6000,drop_last=True )
val_loader = DataLoader(dataset=val_data, batch_size=6000,drop_last=True )

# 2. Hyperparameter Tuning

In [18]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: danish2562022. Use `wandb login --relogin` to force relogin


True

# 3. Model Building for HP tuning

In [19]:
# class BertClassifier(nn.Module):
    
#     def __init__(self,config, embed_dim =1024):
#         super(BertClassifier,self).__init__()
#         self.config = config
#         self.relu = nn.ReLU()
#         self.fc1 =  nn.Linear(embed_dim,config.dim_1)
#         self.fully_connected_layers_1 = nn.ModuleList([nn.Linear(config.dim_1,config.dim_1)
#                                                     for _ in range(config.layer_fc_1)])
        
        
#         self.fc_layer_A = nn.Linear(config.dim_1,config.dim_2)
#         self.fully_connected_layers_2 = nn.ModuleList([nn.Linear(config.dim_2,config.dim_2)
#                                                     for _ in range(config.layer_fc_2)])
        
#         self.dropout_B = nn.Dropout(p = config.dropout_prob_B)
#         self.fc_layer_B = nn.Linear(config.dim_2,config.dim_3)
#         self.fully_connected_layers_3 = nn.ModuleList([nn.Linear(config.dim_3,config.dim_3)
#                                                     for _ in range(config.layer_fc_3)])
#         self.bn = nn.BatchNorm1d(config.dim_3)
#         self.BN_C = config.batch_norm_3
# #         self.fc_layer_C = nn.Linear(config.dim_3,config.dim_4)
#         self.fc_layer_C = nn.Linear(config.dim_2,config.dim_4)
#         self.fully_connected_layers_4 = nn.ModuleList([nn.Linear(config.dim_4,config.dim_4)
#                                                     for _ in range(config.layer_fc_4)])
        
#         self.dropout_C = nn.Dropout(p = config.dropout_prob_C)
#         self.output_layer_1 = nn.Linear(config.dim_4,8)
#         self.output_layer = nn.Linear(8,1)
      
    
#     def forward(self, inputs):
        
#         input = self.fc1(inputs)
#         for i in range(self.config.layer_fc_1):
#             input = self.relu(self.fully_connected_layers_1[i](input))
        
#         input = self.relu(self.fc_layer_A(input))
#         for i in range(self.config.layer_fc_2):
#             input = self.relu(self.fully_connected_layers_2[i](input))
        
        
# #         input = self.relu(self.fc_layer_B(input))
# #         for i in range(self.config.layer_fc_3):
# #             input = self.relu(self.fully_connected_layers_3[i](input))
        
#         if self.BN_C:
#             input = self.bn(input)
          
#         input = self.relu(self.fc_layer_C(input))
#         for i in range(self.config.layer_fc_4):
#             input = self.relu(self.fully_connected_layers_4[i](input))
        
#         input = self.output_layer_1(input)
#         output = self.output_layer(input)

#         return output

In [20]:
class BertClassifier(nn.Module):
    
    def __init__(self,config, embed_dim =1024):
        super(BertClassifier,self).__init__()
        self.relu = nn.ReLU()
        self.config = config
        self.fc1 = nn.Linear(embed_dim,config.dim_1)
        self.fully_connected_layers_1 = nn.ModuleList([nn.Linear(config.dim_1,config.dim_1)
                                                    for _ in range(config.layer_fc_1)])
        
        
    
        self.fc_2 = nn.Linear(config.dim_1,config.dim_2)
        self.fully_connected_layers_2 = nn.ModuleList([nn.Linear(config.dim_2,config.dim_2)
                                                    for _ in range(config.layer_fc_2)]) 
        self.bn2 = nn.BatchNorm1d(num_features=config.dim_2)
        self.fc3 = nn.Linear(config.dim_2,64)
        
        self.fc4 = nn.Linear(64,32)
        self.drop = nn.Dropout(p = 0.2)
        self.fc5 = nn.Linear(32,16)
        self.fc6 = nn.Linear(16,8)
        self.fc7 = nn.Linear(8,1)
    
    def forward(self, inputs):
        
        output = self.relu(self.fc1(inputs))
#         output_1 = self.bn1(output_1)
        for i in range(self.config.layer_fc_1):
            output = self.relu(self.fully_connected_layers_1[i](output))
        output = self.relu(self.fc_2(output))
        for i in range(self.config.layer_fc_2):
            output = self.relu(self.fully_connected_layers_2[i](output))
            
        output  = self.bn2(output)
        output = self.relu(self.fc3(output))
        
        if self.config.dropout:
            output = self.drop(output)
            
        output = self.relu(self.fc4(output))
        output = self.relu(self.fc5(output))
        output = self.relu(self.fc6(output))
        output = self.fc7(output)

        return output

In [21]:
def build_optimizer(network, optimizer,learning_rate, momentum, weight_decay, amsgrad):
    
    
#     if optimizer == "sgd":
#         optimizer_ = optim.SGD(network.parameters(),
#                               lr = learning_rate, momentum = momentum, weight_decay = weight_decay,
#                               )
        
        
    if optimizer == "adam":
        optimizer_ = optim.Adam(network.parameters(),
                               lr = learning_rate, betas = (0.9,0.999), weight_decay = weight_decay,
                               amsgrad = amsgrad)
        
#     elif optimizer == "rms_prop":
#         optimizer_ = optim.RMSprop(network.parameters(),
#                                lr = learning_rate, alpha = 0.99, momentum = momentum,
#                                   weight_decay = weight_decay)
                               
        
    return optimizer_


## 3.1 Hyperparameters Defining

In [22]:
# sweep_config = {
#     'method': 'random'
    
#     }
# metric = {
#     'name': 'val_loss',
#     'goal': 'minimize'   
#     }
# early_terminate = {"type": "hyperband",
#       "min_iter": 3 }

# sweep_config['metric'] = metric 
# sweep_config['early_terminate'] = early_terminate 

# parameters_dict = {
    
#     'layer_fc_1': {
#         'values': [1,2,3,4,5]
#         },
   
#     'dim_1': {
#           'values': [1024,2058]
#         },
    
#     'layer_fc_2': {
#         'values': [1,2]
#         },
#     'dim_2': {
#           'values': [256]
#         },
    
#     'layer_fc_3': {
#         'values': [1,2]
#         },
    
#     'dim_3': {
#           'values': [256]
#         },
        
    
#     'layer_fc_4': {
#         'values': [1,2]
#         },
#     'dim_4': {
#           'values': [256]
#         },
    
#     'layer_fc_5': {
#         'values': [1,2]
#         },
#     'dim_5': {
#           'values': [256]
#         },
    
#     'layer_fc_6': {
#         'values': [1]
#         },
#     'dim_6': {
#           'values': [256]
#         },
    
#     'batch_norm_3': {
#           'values': [True,False]
#         },
    
#     'dropout_prob_C': {
#           'values': [0,0.25,0.5]
#         },
   
#     'dropout_prob_B': {
#           'values': [0,0.25,0.5]
#         },
    
#     'optimizer': {
#           'values': ['adam','sgd']   #
#         }
#     ,
#     'learning_rate': {
#             'distribution': 'uniform',
#             'min': 0,
#             'max': 0.01
#         },
    
#     'momentum': {
#           'values': [0.85,0.9,0.95,0.99]
#         },
    
#     'weight_decay': {
#             'distribution': 'uniform',
#             'min': 0,
#             'max': 0.001
#         },
#     'nesterov': {
#           'values': [True, False]
#         },
        
#     'amsgrad': {
#           'values': [True, False]
#         },
    
    
#     'dampening': {
#           'distribution': 'uniform',
#             'min': 0,
#             'max': 0.1
#         },
#     }


# sweep_config['parameters'] = parameters_dict
# parameters_dict.update({
#     'epochs': {
#         'value': 10}
#     })


# import pprint

# pprint.pprint(sweep_config)

In [24]:
sweep_config = {
    'method': 'random'
    
    }
metric = {
    'name': 'val_accuracy',
    'goal': 'maximize'   
    }
early_terminate = {"type": "hyperband",
      "min_iter": 3 }

sweep_config['metric'] = metric 
sweep_config['early_terminate'] = early_terminate 

parameters_dict = {
    
    'layer_fc_1': {
        'values': [1]
        },
   
    'dim_1': {
          'values': [2048,4048]
        },
    
    'layer_fc_2': {
        'values': [1,2]
        },
    'dim_2': {
          'values': [128,256,512,1024]
        },
    
    
    'dropout': {
          'values': [True,False]
        },
   
    
    
    'optimizer': {
          'values': ['adam']   #
        }
    ,
    'learning_rate': {
            'distribution': 'uniform',
            'min': 0.001,
            'max': 0.01
        },
    
    'momentum': {
          'values': [0.85,0.9,0.95,0.99]
        },
    
    'weight_decay': {
            'distribution': 'uniform',
            'min': 0.001,
            'max': 0.01
        },
   
        
    'amsgrad': {
          'values': [True, False]
        },
    
    
    }


sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 30}
    })


import pprint

pprint.pprint(sweep_config)

{'early_terminate': {'min_iter': 3, 'type': 'hyperband'},
 'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'amsgrad': {'values': [True, False]},
                'dim_1': {'values': [2048, 4048]},
                'dim_2': {'values': [128, 256, 512, 1024]},
                'dropout': {'values': [True, False]},
                'epochs': {'value': 30},
                'layer_fc_1': {'values': [1]},
                'layer_fc_2': {'values': [1, 2]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.01,
                                  'min': 0.001},
                'momentum': {'values': [0.85, 0.9, 0.95, 0.99]},
                'optimizer': {'values': ['adam']},
                'weight_decay': {'distribution': 'uniform',
                                 'max': 0.01,
                                 'min': 0.001}}}


# 4. Training

In [25]:
import random
import time
from tqdm import tqdm
loss_fn = nn.BCEWithLogitsLoss()
def train(config, train_dataloader,val_dataloader = None):
    
    best_accuracy = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Start training...\n")
    epochs = config.epochs
    
    model = BertClassifier(config).to(device)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer,learning_rate, momentum, weight_decay, amsgrad = config.optimizer,config.learning_rate, config.momentum, config.weight_decay, config.amsgrad
    optimizer = build_optimizer(model,optimizer,learning_rate, momentum, weight_decay, amsgrad)
    
    
    for epoch_i in range(1,epochs+1):
        
        total_loss = 0
        model.train()
        
        for step,batch in tqdm(enumerate(train_dataloader)):
            
            inputs, b_labels = tuple(t.to(device) for t in batch)
            b_labels = b_labels.reshape((1,2048*10,1)).squeeze(0)
            model.zero_grad()
            logits = model(inputs)
            loss = loss_fn(logits,b_labels.float()) 
            total_loss += loss.item()
            loss.mean().backward()
            optimizer.step()
               
        avg_train_loss = total_loss / len(train_dataloader)
        
    
        if val_dataloader is not None:
                
                val_loss, val_accuracy = evaluate(model, val_dataloader)
                if val_accuracy > best_accuracy:
                    best_accuracy = val_accuracy
                    torch.save({
                        'epoch': epoch_i ,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss_fn,
                        }, 'best_model_trained_fc_v2.pth')
        
        wandb.log({
                    'epoch': epoch_i, 
                    "train_loss": avg_train_loss,
                    "val_loss": val_loss,
                   })
        print(f"Epoch: {epoch_i} | Training Loss: {avg_train_loss}  | Validation Loss: {val_loss}  | Accuracy: {val_accuracy:.2f}")
        with open('result.txt', 'a') as f:
            print(f"Epoch: {epoch_i} | Training Loss: {avg_train_loss}  | Validation Loss: {val_loss}  | Accuracy: {val_accuracy:.2f}", file=f) 
    print("\n")
    wandb.log({"val_accuracy": best_accuracy,
                   })
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")
    

def evaluate(model,val_dataloader):
    
    
    model.eval()
    val_accuracy = []
    val_loss = []
    for batch in tqdm(val_dataloader):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs, b_labels = tuple(t.to(device) for t in batch)
        b_labels = b_labels.reshape((1,6000,1)).squeeze(0)
        with torch.no_grad():
                logits = model(inputs)
        
        loss = loss_fn(logits, b_labels.float())
        val_loss.append(loss.item())
        preds = torch.round(torch.sigmoid(logits))
        
        accuracy = (preds.float() == b_labels.float()).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)
    
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy 

In [26]:
def make(config):
    # Make the data
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print('The code uses GPU...')
    else:
        device = torch.device('cpu')
        print('The code uses CPU!!!')

    
    model = BertClassifier(config).to(device)

    # Make the loss and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer,learning_rate, momentum, weight_decay, amsgrad = config.optimizer,config.learning_rate, config.momentum, config.weight_decay, config.amsgrad
    optimizer = build_optimizer(model,optimizer,learning_rate, momentum, weight_decay, amsgrad)
    
    return model, criterion, optimizer

In [27]:
def model_pipeline(config=None):

    # tell wandb to get started
    with wandb.init(config=config):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config
      
      # make the model, data, and optimization problem
      
      
      
      # and use them to train the model
      train(config, train_loader,val_dataloader = val_loader)
     
      for i,j in test_loader:
          x = i
      wandb.save("model.onnx")

    

# 5. Tuning

In [28]:
sweep_id = wandb.sweep(sweep_config, project="PI__prediction_fc__hyperparameter_tuning_cls_vector")

Create sweep with ID: gyyc8zyb
Sweep URL: https://wandb.ai/danish2562022/PI__prediction_fc__hyperparameter_tuning_cls_vector/sweeps/gyyc8zyb


In [ ]:
wandb.agent(sweep_id, model_pipeline, count=400)

wandb: Agent Starting Run: ls3eepi5 with config:
wandb: 	amsgrad: True
wandb: 	dim_1: 4048
wandb: 	dim_2: 256
wandb: 	dropout: False
wandb: 	epochs: 30
wandb: 	layer_fc_1: 1
wandb: 	layer_fc_2: 1
wandb: 	learning_rate: 0.002146424428806191
wandb: 	momentum: 0.85
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.00263156505842998
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Start training...



11it [00:06,  1.58it/s]
100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.97it/s]


Epoch: 1 | Training Loss: 0.6851957006887957  | Validation Loss: 0.6937920451164246  | Accuracy: 46.75


11it [00:08,  1.31it/s]
100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.01it/s]


...

11it [01:53, 10.36s/it]
100%|████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.64s/it]


Epoch: 17 | Training Loss: 0.503031925721602  | Validation Loss: 0.665891170501709  | Accuracy: 58.83


11it [03:23, 18.50s/it]
100%|████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.65s/it]


Epoch: 18 | Training Loss: 0.5018074160272424  | Validation Loss: 0.7605113983154297  | Accuracy: 54.58


2it [00:26, 14.58s/it]

In [ ]:
val_label

In [ ]:
val_label